# Notebook exercices chinois - Pix Langue

Résumé de ce qu'il y a dans le note book :

exo vocabulaire => FlashCard / QCM 4 distracteurs de meme pos / QCM 4 distracteurs les plus similaire / QCM 4 distracteurs de meme pos + affiche la phrase en langue target

exo phrase => phrase à trous - 4 distractors de meme pos / phrase à trous - 4 distractors les plus similaire / Remettre les mots dans le bon ordre pour traduire une phrase

Pandas DataFrame Version

In [1]:
import spacy,random,requests
from nltk.corpus import wordnet
nlpCh=spacy.load("zh_core_web_md")
nlpFr = spacy.load("fr_core_news_md")

In [2]:
import pandas as pd

In [3]:
data = pd.read_csv("Corpus mot chinois.csv")
data_phonetique = pd.read_csv("Phonetique Chinoise.csv")
data_phrase = pd.read_csv("Phrase.csv")

In [4]:
display(data_phrase)

,French,Hanzi,Pinyin,Niveau
0,Oui,是的,shì de,1
1,Non,不是,bù (shì),1
2,S'il-vous-plaît,请,qĭng,1
3,Merci,谢谢,xièxie,1
4,Je vous en prie,没关系,méiguānxì,1
...,...,...,...,...
995,Qu'est-ce-qu'il fait chaud aujourd'hui!,今天真热呀！,Jīntiān zhēn rè ya!,4
996,Il y aura du brouillard demain,明天会有雾,Míngtiān huì yǒu wù,5
997,Au voleur !,抓小偷！,Zhuā xiǎotōu!,5
998,On m'a volé,我被抢劫了,Wǒ bèi qiǎngjiéle,6


# Exercices vocabulaire

In [5]:
def vocflash2(motchinois):
    index = data[data["Hanzi"] == motchinois].index
    id = index[0]
    print(data["Hanzi"][id],data["Pinyin"][id], " : ", data["FR"][id])
    
vocflash2("菜")

菜 cài  :  plat (aliments)


In [6]:
def voc121(motchinois, niveau):
    
    mot_nlp = nlpCh(motchinois)
    index = data[data["Hanzi"] == motchinois].index
    id = index[0]
    
    voc = data[data["Niveau"] <= niveau]
    
    for i in voc["Hanzi"]:
        test = nlpCh(i)
        if test[0].pos_ != mot_nlp[0].pos_ or test.text == motchinois:
            voc = voc.drop(voc[voc["Hanzi"] == i].index)
    
    voc = voc.reset_index()
    x1, x2, x3 = random.sample(range(1, len(voc)), 3)

    print(motchinois, " : ")
    print(data["FR"][id])
    print(voc["FR"][x1])
    print(voc["FR"][x2])
    print(voc["FR"][x3])

voc121("电影", 2)

电影  : 
film
crayon; CL: 支[zhi1], 枝[zhi1], 杆[gan3]
couleur; CL: 个
maison


In [7]:
#afficher également les phrases qui contiennent le mot en paramètre
def voc18(motchinois, niveau):
    voc121(motchinois, niveau)
    
    display(data_phrase[data_phrase['Hanzi'].str.contains(motchinois, na=False, regex=False)])
        
voc18("电影", 2)

电影  : 
film
plat (aliments)
se reposer; au repos
taxi


,French,Hanzi,Pinyin,Niveau
498,On se retrouve devant le cinéma,我们在电影院前见面,Wǒmen zài diànyǐngyuàn qián jiànmiàn,3
502,Tu as déjà vu ce film?,你看过了这部电影,Nǐ kànguòle zhè bù diànyǐng,3
504,ça sort la semaine prochaine,它（电影）下周上映,Tā (diànyǐng) xià zhōu shàngyìng,2


In [8]:
def voc122(motfr, niveau):
    
    mot_nlp = nlpFr(motfr)
    index = data[data["FR"] == motfr].index
    id = index[0]
    
    voc = data[data["Niveau"] <= niveau]
    
    for i in voc["FR"]:
        test = nlpFr(i)
        if test[0].pos_ != mot_nlp[0].pos_ or test.text == motfr:
            voc = voc.drop(voc[voc["FR"] == i].index)
    
    voc = voc.reset_index()
    x1, x2, x3 = random.sample(range(1, len(voc)), 3)

    print(motfr, " : ")
    print(data["Hanzi"][id], data["Pinyin"][id])
    print(voc["Hanzi"][x1], voc["Pinyin"][x1])
    print(voc["Hanzi"][x2], voc["Pinyin"][x2])
    print(voc["Hanzi"][x3], voc["Pinyin"][x3])

voc122("chat", 1)

chat  : 
猫 māo
岁 suì
老师 lǎo shī
医生 yī shēng


In [9]:
#utilisation de la similarité

def voc16(motfr, niveau):
    
    index = data[data["FR"] == motfr].index
    id = index[0]
    
    mot_nlp = nlpCh(data["Hanzi"][id])

    distractor = []
    voc = data[data["Niveau"] <= niveau]
    
    x1, x2, x3 = 0, 0, 0
    distractor2 = ""
    distractor3 = ""
        
        
    for i in voc["Hanzi"]:
        i_nlp = nlpCh(i)
        
        if mot_nlp.similarity(i_nlp) > x1 and mot_nlp.similarity(i_nlp) != 1:
            x1 = mot_nlp.similarity(i_nlp)
            distractor1 = i_nlp.text

            if x1 > x2:
                x = x2
                x2 = x1
                x1 = x

                pivot = distractor2
                distractor2 = distractor1
                distractor1 = pivot

                if x2 > x3:
                    x = x3
                    x3 = x2
                    x2 = x
                    pivot = distractor3
                    distractor3 = distractor2
                    distractor2 = pivot
            
            
                        
    return(motfr, mot_nlp.text, distractor1, distractor2, distractor3)
    
voc16("chien", 3)

('chien', '狗', '马', '猫', '动物')

# Exercices - phrase

In [10]:
import re

# ici un trou est fait de façon aléatoire

def closeTest1(phrase):
    
    index = data_phrase[data_phrase["French"] == phrase].index
    id = index[0]
    
    phrasechi = nlpCh(data_phrase["Hanzi"][id])
    
    trou = random.randint(0, len(phrasechi) - 3)
    print(phrase, data_phrase["Hanzi"][id])
    phrase_a_trou = re.sub(phrasechi[trou].text, "...", phrasechi.text)
        
    print(phrase, " : ", phrase_a_trou)
    
closeTest1("Qu'est-ce-qu'il fait chaud aujourd'hui!")

Qu'est-ce-qu'il fait chaud aujourd'hui! 今天真热呀！
Qu'est-ce-qu'il fait chaud aujourd'hui!  :  ...真热呀！


In [11]:
#code en erreur, mais repris differement dans la génération, en utilisant les exos de similarité precedemment généré,
#ce qui a permis de gagner bcp de temps sur la génération

import re

def closeTest2(phrase):

    index = data_phrase[data_phrase["French"] == phrase].index
    id = index[0]
    
    phrasefr = nlpFr(data_phrase["French"][id])
    phrasechi = nlpCh(data_phrase["Hanzi"][id])
    
    trou = random.randint(0, len(phrasechi) - 3)
    print(phrasechi.text, phrasechi[trou].text)
    phrase_a_trou = re.sub(phrasechi[trou].text, "...", phrasechi.text)
    
    print(phrasefr.text, " : ", phrase_a_trou)
    
    index = data[data["Hanzi"] == phrasechi[trou].text].index
    id2 = index[0]
    print(id2)
    
    distractor = []
    #print(distractor)
    for t in data["Hanzi"]:
        mot = data["Hanzi"][0]
        if nlpCh(t)[0].pos_ == nlpCh(mot)[0].pos_: 
            distractor.append(t)  
    
    dist1, sim1, dist2, sim2, dist3, sim3 = voc16(distractor, 2)
    print("\n",dist1, " : ", sim1, "\n", dist2, " : ", sim2, "\n", dist3, " : ", sim3)

 # Traduire avec l'aide des mots proposés

In [ ]:
phrase = "明天会有雾"

In [ ]:
#separe les mots et mélange leur ordre
def phrase1(phrase):
    distractor = []
    answer = []
    while distractor == answer:
        for i in phrase:
            answer.append(i)
            distractor.append(i)
        random.shuffle(distractor)

# marquer les frontière entre les mots
transformer les conjug / genre / question / temps / distracteur quantificateur

In [ ]:
#mélange une phras + ajout de distracteur erreur fréquente
#prise en compte des erreurs récurrentes
#是 + ADJ Verb
#是 au lieu de 在

def phrase1(phrase):
    
    index = data_phrase[data_phrase["Hanzi"] == phrase].index
    id = index[0]
    phrase_fr = data_phrase["French"][id]
    phrase_nlp = nlpCh(phrase)
    distractor = []
    for i in phrase_nlp:
        # pos : adv and dep : advmod => verbadj and indicator of the degree
        if i.pos_ == "ADV" and i.dep_ == "advmod":
            print("indicateur de degré : ", i)
            distractor.append("是")
            
            
        #erreur commune : shi au lieu de hen ou shi + hen
        
        if i.text == "在": #pos du compl de lieux et sa dep, peut être mieux
            print("lieux : ", i)
            distractor.append("是")
            
        distractor.append(i)
    random.shuffle(distractor)
    print(phrase_fr)
    print(distractor)
    
phrase1(phrase)

In [ ]:
#experimentation exercice de négation

def negative(phrase):
    
    index = data_phrase[data_phrase["Hanzi"] == phrase].index
    id = index[0]
    
    phrasefr = nlpCh(data_phrase["French"][id])
        
    print(phrasefr)
    print(phrase)
    
negative("你想吃点什么吗？")

In [ ]:
def voc18():
    
    negation_phrase = data_phrase[data_phrase['Hanzi'].str.contains("没" and "不", na=False, regex=False)]
        
voc18()

# "Homophone" et "Homonyme"

In [ ]:
display(data_phonetique)

In [20]:
word = "Zhōngguó"
print(word)
def distractor_pinyin(word):
    
    morpho_distractor = []

    for t in data_phonetique["Pinyin_consonne"]:
        if type(t) == float:
            break
        elif t in word.lower():
            morpho_distractor.append(t)

    for h in data_phonetique["Pinyin_voyelle"]:
        if type(h) == float:
            break
        elif h in word.lower():
            morpho_distractor.append(h)
    print(morpho_distractor)
    
distractor_pinyin("Zhōngguó")

Zhōngguó
['n', 'z', 'zh', 'g', 'h', 'ō', 'ó', 'ōng', 'u', 'uó']


# Erreurs fréquentes

1- Shi avec un adj_verb

In [21]:
from spacy import displacy
doc = nlpCh("一条衣服。")
#doc = nlpCh("你是法国人")

print(doc[1].dep_)
displacy.render(doc)



mark:clf


['PRON',
 'VERB',
 'PUNCT',
 'DET',
 'NOUN',
 'ADP',
 'NOUN',
 'PART',
 'PART',
 'PUNCT']